In [1]:
import os
import uuid

import requests
from todoist_api_python.api import TodoistAPI
from todoist_api_python.endpoints import get_sync_url
from todoist_api_python.headers import create_headers

from quarter_lib.akeyless import get_secrets
TODOIST_TOKEN = get_secrets("todoist/token")
TODOIST_API = TodoistAPI(TODOIST_TOKEN)

HEADERS = create_headers(token=TODOIST_TOKEN)


def upload_file(file_path):
    return requests.post(
        get_sync_url("uploads/add"),
        headers=HEADERS,
        files={
            "file": open(file_path, "rb"),
        },
    ).json()


def add_note_with_attachement(task_id, file_path):
    file = upload_file(file_path)
    return requests.post(
        get_sync_url("sync"),
        headers=HEADERS,
        json={
            "commands": [
                {
                    "type": "note_add",
                    "temp_id": str(uuid.uuid4()),
                    "uuid": str(uuid.uuid4()),
                    "args": {
                        "item_id": task_id,
                        "content": "",
                        "file_attachment": {
                            "file_name": file["file_name"],
                            "file_size": file["file_size"],
                            "file_type": file['file_type'],
                            "file_url": file["file_url"],
                            "resource_type": "audio",
                        },
                    },
                },
            ]
        },
    ).json()


In [20]:
task_id = "6365837907"

In [21]:
add_note_with_attachement(task_id, r"D:\OneDrive\Code\telegram-assistant\notebooks\temp.wav")

{'full_sync': True,
 'sync_status': {'953fc1d8-7ba7-4aa9-849d-bf84dc4aa2a0': 'ok'},
 'sync_token': 'FNx8LVt7I_UaOfK4Q6Vfp4B77kuEqvRp2jW7JlZeLa-NHWEQ0ijwtW-2F2IqTde6RLMiEX2VZQWYSigJADiiAtzCClSUmyIm2M8pQLX1UY7g6A',
 'temp_id_mapping': {'004a0d06-9c29-430c-bdff-a2514b39cf3a': '3176680268'}}

In [11]:
def run_sync_commands(commands):
    return requests.post(
        get_sync_url("sync"),
        headers=HEADERS,
        json={"commands": commands},
    )


def run_todoist_sync_commands(commands):
    for command in commands:
        command["uuid"] = str(uuid.uuid4())
        if not command.get("temp_id"):
            command["temp_id"] = (str(uuid.uuid4()),)
    return run_sync_commands(commands)


In [21]:
command_list = []
generated_temp_id = "_" + str(uuid.uuid4())
command_list.append(
    {
        "type": "item_add",
        "temp_id": generated_temp_id,
        "args": {"content": "test", "description": "testtest"},
    }
)
command_list.append(
    {
        "type": "note_add",
        "args": {"content": "",
                 "item_id": generated_temp_id,
                 "file_attachment":
                     {
                         "file_name": 'Emotional Intelligence Disc07 Daniel Goleman Emotional Intelligence Disc 7 09 EI.7.9-280.mp3',
                         "file_size": 1764044,
                         "file_type": 'audio/mpeg',
                         "file_url": 'https://files.todoist.com/ZYuC3O85m_PYX_v7uu9WwGf-oMTchdPo4xvMPWyZUzX-hnG3ZpHmg84V2a4qLFhG/by/29307554/as/file.mp3',
                     },
                 }
    }
)
result_list = run_todoist_sync_commands(command_list).json()
display(command_list)
result_list

[{'type': 'item_add',
  'temp_id': '_599adecf-e4f3-4142-8135-45e464194fea',
  'args': {'content': 'test', 'description': 'testtest'},
  'uuid': 'dd6afe19-cc9c-42c1-bc78-598bae3768eb'},
 {'type': 'note_add',
  'args': {'content': '',
   'item_id': '_599adecf-e4f3-4142-8135-45e464194fea',
   'file_attachment': {'file_name': 'Emotional Intelligence Disc07 Daniel Goleman Emotional Intelligence Disc 7 09 EI.7.9-280.mp3',
    'file_size': 1764044,
    'file_type': 'audio/mpeg',
    'file_url': 'https://files.todoist.com/ZYuC3O85m_PYX_v7uu9WwGf-oMTchdPo4xvMPWyZUzX-hnG3ZpHmg84V2a4qLFhG/by/29307554/as/file.mp3'}},
  'uuid': '6981d8df-8337-482c-a4c5-8ed160316768',
  'temp_id': ('36cf8ba3-07eb-418c-8529-2612c1188e48',)}]

{'full_sync': True,
 'sync_status': {'6981d8df-8337-482c-a4c5-8ed160316768': 'ok',
  'dd6afe19-cc9c-42c1-bc78-598bae3768eb': 'ok'},
 'sync_token': 'fw7p9dIb8wZabdDHEBlnSawqvsQ3HmnMFDMKh4hxL-0Y4K8OknwvxHkf5xD3L82-38iLHYTDiYWEMku2aznYrigCvbcl8FIOVoM04I41_OIyFrUP',
 'temp_id_mapping': {"['36cf8ba3-07eb-418c-8529-2612c1188e48']": '3295827092',
  '_599adecf-e4f3-4142-8135-45e464194fea': '6964887153'}}